# Ising spin glass model

## 1D

We study the Hamiltonian of the transverse-field Ising spin glass model without periodic boundary conditions and transverse field `g` set to 0.4.

In [1]:
import Random
using CondensedMatterSOS
@spin σ[1:2]
function hamiltonian(σ)
    Random.seed!(1) # We set the seed so that the results of this example are reproducible.
    return ising_glass_hamiltonian(σ, 0.4, false)
end
hamiltonian(σ)

(0.2972879845354616 + 0.0im)σˣ₁σˣ₂ + (-0.4 + 0.0im)σᶻ₁ + (-0.4 + 0.0im)σᶻ₂

Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

In [2]:
using CSDP
solver = optimizer_with_attributes(
    () -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), CSDP.Optimizer()),
    MOI.Silent() => true
)

MathOptInterface.OptimizerWithAttributes(Main.##254.var"#1#2"(), Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.Silent() => true])

We can compute a lower bound `-0.853452` to the ground state energy as follow:

In [3]:
function hamiltonian_energy(N, maxdegree, solver; kws...)
    @spin σ[1:N]
    H = hamiltonian(σ)
    energy(H, maxdegree, solver; kws...)
end
bound, gram, ν = hamiltonian_energy(2, 2, solver, sparsity = NoSparsity())
bound

-0.8534519037352409

We can see that the moment matrix uses all monomials:

In [4]:
ν.basis.monomials

7-element Array{CondensedMatterSOS.SpinMonomial,1}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

Using term sparsity with block/cluster completion, we get the same bound:

In [5]:
bound, gram, ν = hamiltonian_energy(2, 2, solver)
bound

-0.8534519037605935

But with a smaller basis:

In [6]:
[M.basis.monomials for M in ν.sub_moment_matrices]

2-element Array{Array{CondensedMatterSOS.SpinMonomial,1},1}:
 [σᶻ₁, σᶻ₂, 1]
 [σˣ₁, σʸ₁, σˣ₂, σʸ₂]

Using term sparsity with chordal completion, we get a smaller bound:

In [7]:
bound, gram, ν = hamiltonian_energy(2, 2, solver, sparsity = MonomialSparsity(ChordalCompletion()))
bound

-1.0972879866755514

But with an even smaller basis:

In [8]:
[M.basis.monomials for M in ν.sub_moment_matrices]

5-element Array{Array{CondensedMatterSOS.SpinMonomial,1},1}:
 [σˣ₂, σʸ₂]
 [σᶻ₂, 1]
 [σᶻ₁, 1]
 [σˣ₁, σˣ₂]
 [σˣ₁, σʸ₁]

## 2D

We now study the same model but on a square lattice.

In [9]:
@spin σ[1:2, 1:2]
hamiltonian(σ)

(0.2972879845354616 + 0.0im)σˣ₁₋₁σˣ₂₋₁ + (-0.5976344767282311 - 0.0im)σˣ₁₋₁σˣ₁₋₂ + (-0.01044524463737564 - 0.0im)σˣ₂₋₁σˣ₂₋₂ + (0.3823959677906078 + 0.0im)σˣ₁₋₂σˣ₂₋₂ + (-0.4 + 0.0im)σᶻ₁₋₁ + (-0.4 + 0.0im)σᶻ₂₋₁ + (-0.4 + 0.0im)σᶻ₁₋₂ + (-0.4 + 0.0im)σᶻ₂₋₂

We can compute a lower bound `-1.990513` to the ground state energy as follow:

In [10]:
function hamiltonian_energy(N, M, maxdegree, solver; kws...)
    @spin σ[1:N, 1:M]
    H = hamiltonian(σ)
    energy(H, maxdegree, solver; kws...)
end
bound, gram, ν = hamiltonian_energy(2, 2, 2, solver, sparsity = NoSparsity())
bound

-1.9905121018001712

We can see that the moment matrix uses all monomials:

In [11]:
ν.basis.monomials

13-element Array{CondensedMatterSOS.SpinMonomial,1}:
 1
 σˣ₁₋₁
 σʸ₁₋₁
 σᶻ₁₋₁
 σˣ₂₋₁
 σʸ₂₋₁
 σᶻ₂₋₁
 σˣ₁₋₂
 σʸ₁₋₂
 σᶻ₁₋₂
 σˣ₂₋₂
 σʸ₂₋₂
 σᶻ₂₋₂

Using term sparsity with block/cluster completion, we get the same bound:

In [12]:
bound, gram, ν = hamiltonian_energy(2, 2, 2, solver)
bound

-1.9905121009002253

But with a smaller basis:

In [13]:
[M.basis.monomials for M in ν.sub_moment_matrices]

2-element Array{Array{CondensedMatterSOS.SpinMonomial,1},1}:
 [σᶻ₁₋₁, σᶻ₂₋₁, σᶻ₁₋₂, σᶻ₂₋₂, 1]
 [σˣ₁₋₁, σʸ₁₋₁, σˣ₂₋₁, σʸ₂₋₁, σˣ₁₋₂, σʸ₁₋₂, σˣ₂₋₂, σʸ₂₋₂]

Using term sparsity with chordal completion, we get a smaller bound:

In [14]:
bound, gram, ν = hamiltonian_energy(2, 2, 2, solver, sparsity = MonomialSparsity(ChordalCompletion()))
bound

-2.655703628431353

But with an even smaller basis:

In [15]:
[M.basis.monomials for M in ν.sub_moment_matrices]

9-element Array{Array{CondensedMatterSOS.SpinMonomial,1},1}:
 [σᶻ₁₋₂, σᶻ₂₋₂, 1]
 [σˣ₁₋₂, σʸ₁₋₂, σˣ₂₋₂]
 [σˣ₂₋₁, σˣ₁₋₂, σˣ₂₋₂]
 [σˣ₁₋₁, σˣ₂₋₁, σˣ₁₋₂]
 [σˣ₂₋₂, σʸ₂₋₂]
 [σᶻ₂₋₁, 1]
 [σˣ₂₋₁, σʸ₂₋₁]
 [σᶻ₁₋₁, 1]
 [σˣ₁₋₁, σʸ₁₋₁]

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*